# Comparison of observed and simulated ARI wind speeds

This notebook plots the average recurrence interval (ARI) wind speeds based on observed wind speeds corresponding to the passage of TCs (within 200 km of a station). It adds a plot of the fitted ARI wind speeds from a TCRM simulation.

In [1]:
%matplotlib inline

import os
import io
import sys
from os.path import join as pjoin
import numpy as np
import matplotlib.pyplot as plt

import pandas as pd
import geopandas as gpd
from datetime import datetime



from extremes import returnLevels, empReturnPeriod
from distributions import fittedPDF

# Import widgets for interactive notebook
from ipywidgets import interact, fixed
import ipywidgets as widgets

import seaborn as sns
sns.set_context("poster")
sns.set_style("whitegrid")

In [2]:
def loadObservations(stnId):
    names = ['recid', 'stnId', 'datetime', 'gust',
             'direction', 'quality', 'cycName']
    
    filename = pjoin(obsPath, "bom_{0:06d}.csv".format(stnId))
    try:
        obsdf = pd.read_csv(filename, skiprows=1, names=names,
                            parse_dates=[2], infer_datetime_format=True)
    except FileNotFoundError:
        print("No data file for stnId: {0}".format(stnId))
        return None
    return obsdf

def getStationDates(stnId):
    startYear = stndf.loc[stnId]['stnDataStart']
    endYear = stndf.loc[stnId]['stnDataEnd']
    numYears = endYear - startYear + 1
    return numYears

In [3]:
STNTYPES = [('st', 'S2'), ('stnId', 'i'), ('stnDistCode', 'S4'), ('stnName', 'S'), 
            ('stnDateOpen', 'S10'), ('stnDateClosed', 'S10'), ('stnLat', 'f8'), 
            ('stnLon', 'f8'), ('method', 'S15'), ('state', 'S3'), 
            ('stnElevation', 'f8'), ('baroElev', 'i'), ('stnWMONumber', 'i'), ('stnDataStart', 'i'), 
            ('stnDataEnd', 'i'), ('blank', 'S3'), ('percentcomplete', 'f8'), ('pcqualy', 'f8'), 
            ('pcqualn', 'f8'), ('pcqualw', 'f8'), ('pcquals', 'f8'), ('pcquali', 'f8'), ('end', 'S1')]
STNCONVERT = {'stnName' : str.rstrip}

Start with loading the observation station information. This is from the daily maximum wind gust dataset (Geosciene Australia eCat #110561), starting with the station details file.

In [4]:
obsPath = "C:/WorkSpace/data/derived/tcobs/daily"
stationFilePath = "C:/WorkSpace/data/raw/daily_max_wind_gust/"
stnfile = pjoin(stationFilePath, "DC02D_StnDet_999999999425050.txt")

stndf = pd.read_csv(stnfile, parse_dates=[4, 5],
                        usecols=(1,2,3,4,5,6,7,9,10,12,13,14,16), 
                        names = np.dtype(STNTYPES).names,
                        skiprows=1, engine='python', index_col='stnId', 
                        converters=STNCONVERT)
stationNameList = list(stndf['stnName'])
stndf.head(10)

,stnDistCode,stnName,stnDateOpen,stnDateClosed,stnLat,stnLon,state,stnElevation,stnWMONumber,stnDataStart,stnDataEnd,percentcomplete
stnId,,,,,,,,,,,,
1006,01,WYNDHAM AERO,01/1951,,-15.5100,128.1503,WA,3.8,95214,2003,2017,96
1007,01,TROUGHTON ISLAND,09/1956,,-13.7542,126.1485,WA,6.0,94102,2008,2017,94
1009,01,KURI BAY,08/1961,09/2012,-15.4875,124.5222,WA,12.0,,2001,2001,100
1019,01,KALUMBURU,11/1997,,-14.2964,126.6453,WA,23.0,94100,2003,2017,93
1020,01,TRUSCOTT,01/1944,,-14.0900,126.3867,WA,51.0,95101,2004,2017,93
2012,02,HALLS CREEK METEOROLOGICAL OFFICE,01/1944,,-18.2292,127.6636,WA,422.0,99201,1962,2017,94
2056,02,KUNUNURRA AERO,09/1971,,-15.7814,128.7100,WA,44.0,94216,1994,2017,97
2064,02,ARGYLE AERODROME,01/1986,,-16.6381,128.4517,WA,164.0,94217,2002,2017,99
3003,03,BROOME AIRPORT,01/1939,,-17.9475,122.2353,WA,7.4,94203,1941,2017,99


Now load a shape file that contains the observed stations joined with the TCRM simulation locations. Note in this dataframe, we need to add an index, and so we index by both the location id number (TCRM simulation locations) *and* the station number (observations).

In [14]:
locationFilePath = "C:/WorkSpace/data/derived/tcobs/merged.shp"
locdf = gpd.read_file(locationFilePath)
locdf = locdf.set_index(["locId", 'stnId'])
locationNameList = list(locdf['Place'])
locdf.head(10)

,,Cou,Elevation,ICAO,Latitude,Longitude,Place,St,WMO,geometry,percentcom,...,stnDataEnd,stnDataSta,stnDateClo,stnDateOpe,stnDistCod,stnElevati,stnLat,stnLon,stnName,stnWMONumb
locId,stnId,,,,,,,,,,,,,,,,,,,,,
10210,300001,AAR,10,----,-67.60000,62.86667,Mawson,--,89564,POINT (62.86667 -67.59999999999999),0,...,2017,1996,None,01/1954,300,9.9,-67.6017,62.8753,MAWSON,89564
10213,300000,AAR,18,----,-68.58333,77.96667,Davis,--,89571,POINT (77.96666999999999 -68.58333),0,...,2017,1994,None,01/1957,300,18.0,-68.5744,77.9672,DAVIS,89571
10219,300017,AAR,40,----,-66.28333,110.51667,Casey,--,89611,POINT (110.51667 -66.28333000000001),0,...,2017,1998,None,02/1989,300,40.0,-66.2825,110.5231,CASEY,89611
10626,1007,AUS,6,----,-13.75000,126.15000,Troughton Island W. A.,--,94102,POINT (126.15 -13.75),0,...,2017,1990,None,09/1956,01,6.0,-13.7542,126.1485,TROUGHTON ISLAND,94102
10627,200784,AUS,4,----,-14.11667,123.53333,Browse Island Aws,--,94103,POINT (123.53333 -14.11667),0,...,2017,2013,None,01/1969,200,3.7,-14.1089,123.5472,BROWSE ISLAND,94103
10628,14314,AUS,3,----,-14.03333,121.75000,Scott Reef Aws,--,94105,POINT (121.75 -14.03333),0,...,2017,2013,None,06/2013,14GA,17.5,-12.6099,131.0474,NOONAMAH AIRSTRIP,94105
10630,14277,AUS,4,----,-12.63333,130.36667,Dum In Mirrie Aws,--,94116,POINT (130.36667 -12.63333),0,...,2017,2000,None,04/1994,14GA,3.5,-12.6350,130.3725,DUM IN MIRRIE AIRSTRIP,94116
10632,14142,AUS,12,----,-11.40000,130.41667,Garden Point,--,94119,POINT (130.41667 -11.4),0,...,2017,2006,None,01/1963,14GA,18.3,-11.4021,130.4217,PIRLANGIMPI AIRPORT,94119
10635,200731,AUS,14,----,-11.78333,130.01667,Bathurst Island Aws Cape Fourcroy,--,94122,POINT (130.01667 -11.78333),0,...,2017,2000,None,07/1971,14GA,6.5,-11.7628,130.0300,POINT FAWCETT,94122


Indexing using the `locId` first, then selecting the `index` attribute returns the `stnId`, which is used to load the observed data

In [19]:
locdf.loc[10856].index[0]

40842

Using multiple indexes means you can directly access the attribute using the DataFrame.`loc` method

In [20]:
locdf.loc[10856, 40842]

Cou                               AUS
Elevation                           4
ICAO                             YBBN
Latitude                     -27.3833
Longitude                       153.1
Place           Brisbane Airport M. O
St                                 --
WMO                             94578
geometry      POINT (153.1 -27.38333)
percentcom                          0
state                             QLD
stnDataEnd                       2017
stnDataSta                       1995
stnDateClo                       None
stnDateOpe                    12/1992
stnDistCod                         40
stnElevati                        4.5
stnLat                       -27.3917
stnLon                        153.129
stnName                 BRISBANE AERO
stnWMONumb                      94578
Name: (10856, 40842), dtype: object

The parameters of the fitted distribution are contained in another data file, and this is indexed using the TCRM location id number.

In [21]:
paramFile = "C:/WorkSpace/data/derived/tc/tcha/parameters.csv"
paramNames = ['locId', "locName", "it_scale", "it_shape", "it_thresh", 
              "it_rate", "gpd_rate", "gpd_shape", "gpd_thresh", "gpd_scale"]
gpddf = pd.read_csv(paramFile, names=paramNames, skiprows=1, index_col='locId')
gpddf.head(40)

,locName,it_scale,it_shape,it_thresh,it_rate,gpd_rate,gpd_shape,gpd_thresh,gpd_scale
locId,,,,,,,,,
10605,Nadzab,0.000000,0.000000,0.000000,0.001023,0.000285,-0.064924,20.561479,5.074588
10593,Madang,-0.057574,28.696775,4.071263,0.000034,0.000285,-0.119735,18.843449,5.356575
10588,Koinambe,-0.061601,34.191239,3.639061,0.000008,0.000285,-0.116861,20.041780,5.002796
10587,Mount Hagen,-0.066907,26.584459,4.366419,0.000083,0.000285,-0.062477,21.411610,4.436294
10606,Erave,-0.041564,25.785780,4.228511,0.000161,0.000285,-0.059708,23.213079,4.513520
10589,Goroka,-0.098329,39.170346,4.277844,0.000004,0.000285,-0.101515,21.397353,5.185086
10590,Kundiawa,-0.095997,33.728931,4.539020,0.000016,0.000285,-0.075342,21.635276,4.814428
10594,Aiyura,-0.170982,39.656230,4.918647,0.000004,0.000285,-0.086113,21.925409,5.221558
10598,Bulolo,-0.041065,31.714521,4.435643,0.000070,0.000285,-0.104410,24.169427,5.716989


In [22]:
def plotObservedHazard(locId, ax):
    obsdf = loadObservations(locId)
    if obsdf is None:
        return ax
    numYears = getStationDates(locId)
    data = np.zeros(int(numYears * 365.25))
    wspd = np.sort(np.array(obsdf['gust']))*1.114 # Include conversion to 0.2 second wind gust
    data[-len(wspd):] = wspd
    emprp = empReturnPeriod(data)
    
    ax.scatter(emprp[emprp > 1], data[emprp > 1], s=50,
                color='k', marker='x', label="Empirical ARI")
    return ax
    """
    ax.set_xscale("log")
    ax.axhline(45.6, c='lime', linestyle='--', linewidth=2)#, label='Cat 3')
    ax.axhline(62.5, c='darkorange', linestyle='--', linewidth=2)#, label='Cat 4')
    ax.axhline(77.8, c='darkred', linestyle='--', linewidth=2)#, label='Cat 5')
    ax.text(2000, 45.6, 'Cat 3', ha='center')
    ax.text(2000, 62.5, 'Cat 4', ha='center')
    ax.text(2000, 77.8, 'Cat 5', ha='center')
    ax.legend(loc=2)
    ax.set_ylim((0, 100))
    ax.set_xlim((0, 100))
    ax.set_yticks(np.arange(0, 101, 10))
    ax.set_xlim((1, 1000))
    ax.set_ylabel('Wind speed (m/s)')
    ax.set_xlabel('Average recurrence interval (years)')
    ax.grid(which='major', linestyle='-')
    ax.grid(which='minor', linestyle='--', linewidth=1)
    return ax
    """

In [23]:
def plotFittedHazard(gpd_params, ax):
    """
    Plot a fitted distribution, with approximate 90% confidence interval
    and empirical return period values.

    :param data: :class:`numpy.ndarray` of observed data values.
    :param float mu: Selected threshold value.
    :param float xi: Fitted shape parameter.
    :param float sigma: Fitted scale parameter.
    :param str title: Title string for the plot.
    :param str figfile: Path to store the file (includes image format)

    """
    
    rp = np.array([1, 2, 5, 10, 20, 50, 100, 200,
                   500, 1000, 2000, 5000, 10000])
    mu, xi, sigma, rate = gpd_params
    rval = returnLevels(rp, mu, xi, sigma, rate)

    ax.semilogx(rp, rval, label="Fitted hazard curve")
    return ax


    

In [26]:
def loadParameters(locationName):
    locId = locdf.index[locationNameList.index(locationName)][0]   
    try:
        stnId = locdf.loc[locId].index[0]
    except KeyError:
        print("No index for given location id: {0}".format(locId))
    else:
        stnName = locdf.loc[locId, stnId]['Place']

        stnObsFile = pjoin(obsPath, "bom_{0:06d}.csv".format(stnId))
        if os.path.exists(stnObsFile):
            print("Observation file exists for {0}".format(stnName))
        else:
            print("No observations for {0}".format(stnName))
            
    if locId in gpddf.index.values:
        gpd_rate = gpddf.loc[locId]['gpd_rate']
        gpd_shape = gpddf.loc[locId]['gpd_shape']
        gpd_scale = gpddf.loc[locId]['gpd_scale']
        gpd_thresh = gpddf.loc[locId]['gpd_thresh']
        
        fig, ax = plt.subplots(1, 1, figsize=(9,7))
        plotFittedHazard((gpd_thresh, gpd_shape, gpd_scale, gpd_rate), ax)
        plotObservedHazard(stnId, ax)
        
        title_str = (stnName)  # + "\n" +
                 #r"$\mu$ = {0:.3f}, $\xi$ = {1:.5f}, $\sigma$ = {2:.4f}".
                 #format(mu, xi, sigma))
        ax.set_title(title_str)
        ax.set_ylim((0, 100))
        ax.set_yticks(np.arange(0, 101, 10))
        ax.set_xlim((1, 10000))
        ax.set_ylabel('Wind speed (m/s)')
        ax.set_xlabel('Average recurrence interval (years)')
        ax.grid(which='major', linestyle='-')
        ax.grid(which='minor', linestyle='--', linewidth=1)
        ax.axhline(45.6, c='lime', linestyle='--', linewidth=2)#, label='Cat 3')
        ax.axhline(62.5, c='darkorange', linestyle='--', linewidth=2)#, label='Cat 4')
        ax.axhline(77.8, c='darkred', linestyle='--', linewidth=2)#, label='Cat 5')
        ax.text(20000, 45.6, 'Cat 3', ha='center')
        ax.text(20000, 62.5, 'Cat 4', ha='center')
        ax.text(20000, 77.8, 'Cat 5', ha='center')
        ax.legend(loc=2)
        plt.show()
    else:
        print("No index in GPD parameter file for {0}".format(locId))
        
    

In [27]:
interact(loadParameters, locationName=locationNameList)

<function __main__.loadParameters>

In [ ]:
print(locdf.index[locationNameList.index("DARWIN AIRPORT")][0] )
